# Data Understanding & Warehouse Model Design

## Project: E-Commerce Customer Retention Analysis

### Objective
The objective of this notebook is to understand the raw Olist e-commerce dataset and design an analytical data model suitable for customer retention and behavior analysis.

This notebook focuses on:

- Understanding table structures
- Validating primary keys and data grain
- Identifying customer identity challenges
- Designing a star-schema inspired analytical model
- Defining modeling assumptions before transformations

This approach follows Analytics Engineering best practices: model first, transform second.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

## Load Raw Tables

In [4]:
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")

## Table Shapes

In [5]:
tables = {
    "orders": orders,
    "customers": customers,
    "order_items": order_items,
    "payments": payments,
    "reviews": reviews
}

for name, df in tables.items():
    print(f"{name}: {df.shape}")

orders: (99441, 8)
customers: (99441, 5)
order_items: (112650, 7)
payments: (103886, 5)
reviews: (99224, 7)


In [6]:
# Raw table preview
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


## Primary Key Validation

In [7]:
print("Orders - unique order_id:", orders["order_id"].nunique())
print("Orders - total rows:", len(orders))

Orders - unique order_id: 99441
Orders - total rows: 99441


In [8]:
print("Customers - unique customer_id:", customers["customer_id"].nunique())
print("Customers - total rows:", len(customers))

Customers - unique customer_id: 99441
Customers - total rows: 99441


#### Primary Key Observations

- `order_id` appears to be unique in the orders table.
- `customer_id` appears to be unique in the customers table.

This suggests:

- Grain of `orders` table: 1 row per order.
- Grain of `customers` table: 1 row per customer_id.

Next step: validate customer identity logic.

## Customer Identity Issue

In [9]:
print("Unique customer_id:", customers["customer_id"].nunique())
print("Unique customer_unique_id:", customers["customer_unique_id"].nunique())

Unique customer_id: 99441
Unique customer_unique_id: 96096


#### Customer Identity Insight

Important finding:

- `customer_id` represents a purchase-level identifier.
- `customer_unique_id` represents the actual unique customer.

This means:

If a customer makes multiple purchases, they may appear with different `customer_id` values but the same `customer_unique_id`.

For retention analysis, we MUST use `customer_unique_id`.

Failing to do so would inflate customer counts and distort retention metrics.

## Order Status Distribution

In [10]:
orders["order_status"].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

#### Order Status Observation

Orders contain multiple statuses (e.g., delivered, canceled, shipped).

For retention and revenue analysis, we will likely:

- Focus on delivered orders
- Exclude canceled orders
- Validate payment presence

This decision will be formalized in the transformation layer.

## Revenue Source Validation

In [11]:
print("Payments - total rows:", len(payments))
print("Unique order_ids in payments:", payments["order_id"].nunique())

Payments - total rows: 103886
Unique order_ids in payments: 99440


## Review Coverage Check

In [12]:
print("Unique order_ids in reviews:", reviews["order_id"].nunique())

Unique order_ids in reviews: 98673


## Join Strategy Design

To build a fact table at order level, we will:

1. Start from `orders`
2. Join `customers` to retrieve `customer_unique_id`
3. Join `payments` to calculate total order value
4. Join `reviews` to retrieve review score

Join Keys:

- orders.customer_id → customers.customer_id
- orders.order_id → payments.order_id
- orders.order_id → reviews.order_id

Target grain:

Final fact table → 1 row per order_id

## Proposed Data Model

### Fact Table: fact_orders
Grain: 1 row per order

Columns:
- order_id
- customer_unique_id
- order_purchase_timestamp
- order_value
- review_score
- delivery_delay_days
- order_year
- order_month

---

### Dimension Table: dim_customers
Grain: 1 row per customer_unique_id

Columns:
- customer_unique_id
- customer_city
- customer_state

---

### Analytical Table: customer_metrics
Grain: 1 row per customer

Columns:
- first_purchase_date
- last_purchase_date
- total_orders
- total_revenue
- avg_order_value
- tenure_days
- repeat_flag

## Modeling Assumptions

1. Revenue will be calculated using payment_value from payments table.
2. Retention analysis will use customer_unique_id.
3. Only valid completed transactions (likely delivered orders) will be included.
4. Delivery delay will be calculated as:
   actual_delivery_date - estimated_delivery_date
5. Missing reviews will not exclude an order from analysis.

These assumptions will be validated and implemented in the transformation notebook.

## Next Steps

In the next notebook:

- Clean raw tables (date parsing, null handling)
- Filter relevant order statuses
- Construct fact_orders table
- Validate grain consistency after joins
- Begin feature engineering

This notebook focused on modeling design before transformation,
following Analytics Engineering best practices.